# Simulação - Exemplo 1

Neste Notebook iremos simular o Exemplo 1 do 2º Trabalho de Inferência Estatística da FGV - EMAp ministrada pelo professor Luiz Max Carvalho.

### Enunciado

Suponha que temos duas moedas, Moeda 1 e Moeda 2 de modo que $Pr(\textit{Cara }|\textit{ Moeda} = 1) = p_1$ e $Pr(\textit{Cara }|\textit{ Moeda} = 2) = p_2$; Suponha que agora façamos o seguinte experimento:
  * selecionamos uma moeda aleatoriamente com probabilidade $\frac{1}{2}$;
      
  * lançamos a moeda selecionada $m$ vezes;
            
  * repetimos (i) e (ii) $n$ vezes.
        
Podemos representar os dados advindos desse experimento como
\begin{equation*}
    \begin{array}{ccccc}
        X_{11} & \dots & X_{1m} & & M_1 \\
        X_{21} & \dots & X_{2m} & & M_2 \\
        \vdots & \ddots & \vdots & & \vdots \\
        X_{n1} & \dots & X_{nm} & & M_n 
    \end{array}
\end{equation*}
onde $X_{ij}$ é a variável de Bernoulli que representa o resultado do $j$-ésimo lançamento da moeda na $i$-ésima rodada e $M_i \in \{1, 2\}$ é a variável aleatória que guarda a informação sobre qual moeda foi lançada na $i$-ésima rodada do experimento.
    
Desenvolva um esquema EM para obter o EMV de $\theta = (p_1, p_2)$ quando desconhecemos os valores de $M_i$, isto é, quando não sabemos que moeda foi escolhida em cada uma das $n$ rodadas.

Abaixo temos os import's utilizados para realizar a simulação, a definição de $\theta = (p_1, p_2)$, $n$, $m$ e os lançamentos de moedas, que são os dados que temos em mãos.

In [1]:
import random as rd
import math
import numpy as np
from scipy.optimize import minimize

p1 = 0.10 # prababilidade de sair cara na moeda 1
p2 = 0.70 # prababilidade de sair cara na moeda 2
n = 50 # repetições do processo
m = 400 # lançamentos de moedas por iteração
resultados = np.zeros((n, m))

for i in range(n):
    if rd.random() < 0.5:
        for j in range(m):
            if rd.random() < p1:
                resultados[i][j] = 1
    else:
        for j in range(m):
            if rd.random() < p2:
                resultados[i][j] = 1


Feito isso, definimos a função de log-verossimilhança negativa, uma vez que maximizar a função de log-verossimilhança é o mesmo que minimizar tal função, assim, podemos nos valer da função minimize da biblioteca scipy.optimize.

In [2]:
def log_verossimilhanca_negativa(theta, resultados):
    m = len(resultados[0])
    n = len(resultados)
    soma = 0
    for i in range(n):
        parcela_M1 = 1
        parcela_M2 = 1
        for j in range(m):
            if resultados[i][j] == 1:
                parcela_M1 *= theta[0]
                parcela_M2 *= theta[1]
            else:
                parcela_M1 *= (1 - theta[0])
                parcela_M2 *= (1 - theta[1])
        parcela = (parcela_M1 + parcela_M2)/2
        soma += math.log(parcela)
    
    return -soma

Agora, definimos o loop do algoritmo EM.

Note que o passo E equivale a utilizar $\theta^{(p)}$ na função definida acima, enquanto o passo M irá, nesse caso, minimizar a função log_verossimilhanca_negativa.

In [3]:
def loop_EM(theta, resultados):
    results = minimize(log_verossimilhanca_negativa, theta, args = resultados, bounds = ((0.1, 0.9), (0.1, 0.9)), method = 'SLSQP')
    theta = results.x
    result_ant = results.fun
    diff = result_ant - 0
    while diff > 0.000000000001:
        results = minimize(log_verossimilhanca_negativa, theta, args = resultados, bounds = ((0.1, 0.9), (0.1, 0.9)), method = 'SLSQP')
        theta = results.x
        diff = result_ant - results.fun
        result_ant = results.fun

    return results


Feito isso, podemos utilizar o algoritmo para estimar $\theta$ a partir de diferentes $\theta^{(0)}$:

In [4]:
theta = [0.3, 0.9]
print('Theta inicial:', theta)
results = loop_EM(theta, resultados)

print('Theta final:', results.x)
print('Função log-verossimilhança negativa:', results.fun)
print()

theta = [0.5, 0.5]
print('Theta inicial:', theta)
results = loop_EM(theta, resultados)

print('Theta final:', results.x)
print('Função log-verossimilhança negativa:', results.fun)
print()

theta = [0.1, 0.7]
print('Theta inicial:', theta)
results = loop_EM(theta, resultados)

print('Theta final:', results.x)
print('Função log-verossimilhança negativa:', results.fun)
print()

print('Theta usado para gerar os dados:', [p1, p2])
print('Função log-verossimilhança negativa:', log_verossimilhanca_negativa([p1, p2], resultados))

Theta inicial: [0.3, 0.9]
Theta final: [0.10349999 0.6984    ]
Função log-verossimilhança negativa: 9483.855228185645

Theta inicial: [0.5, 0.5]
Theta final: [0.40095055 0.40095055]
Função log-verossimilhança negativa: 13467.899582980837

Theta inicial: [0.1, 0.7]
Theta final: [0.10349999 0.69839999]
Função log-verossimilhança negativa: 9483.855228185643

Theta usado para gerar os dados: [0.1, 0.7]
Função log-verossimilhança negativa: 9484.589739464376


Note que com $\theta^{(0)} = (0.5, 0.5)$ o $\theta$ final não chegou próximo ao $\hat{\theta}$, o que nos mostra que nem sempre o Algoritmo EM vai para o ponto excelente, mas note que ele ele tende a um ponto de excelência local:

In [5]:
print('Theta:', [0.5, 0.5])
print('Função log-verossimilhança negativa:', log_verossimilhanca_negativa([0.5, 0.5], resultados))
print()
print('Theta:', [0.41, 0.41])
print('Função log-verossimilhança negativa:', log_verossimilhanca_negativa([0.41, 0.41], resultados))
print()
print('Theta:', [0.39, 0.39])
print('Função log-verossimilhança negativa:', log_verossimilhanca_negativa([0.39, 0.39], resultados))

Theta: [0.5, 0.5]
Função log-verossimilhança negativa: 13862.9436111989

Theta: [0.41, 0.41]
Função log-verossimilhança negativa: 13471.293201425553

Theta: [0.39, 0.39]
Função log-verossimilhança negativa: 13472.923112787748


Note que o valor final de $\theta$ quando $\theta^{(0)} = (0.5, 0.5)$ ainda minimizou a a função de log-verossimilhança negativa